In [17]:
from nba_api.stats.static import teams
from nba_api.live.nba.endpoints import scoreboard
from utils import fetch_data, fetch_player_stats
from nba_api.stats.endpoints import teamdetails, teamyearbyyearstats, commonteamroster, teamgamelog
import pandas as pd
import plotly.express as px
from datetime import datetime
import re

In [3]:
team_id = 1610612738 # celtics https://www.nba.com/stats/team/1610612738

In [4]:
previous_season = datetime.now().year - 1
game_log = fetch_data(teamgamelog.TeamGameLog, team_id=team_id, season=previous_season)
print(game_log)

2024-10-13 17:58:47.818 
  command:

    streamlit run C:\Users\ydebray\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-10-13 17:58:47.818 No runtime found, using MemoryCacheStorageManager


       Team_ID     Game_ID     GAME_DATE      MATCHUP WL   W   L  W_PCT  MIN   
0   1610612738  0022301186  APR 14, 2024  BOS vs. WAS  W  64  18  0.780  240  \
1   1610612738  0022301173  APR 12, 2024  BOS vs. CHA  W  63  18  0.778  240   
2   1610612738  0022301167  APR 11, 2024  BOS vs. NYK  L  62  18  0.775  240   
3   1610612738  0022301148  APR 09, 2024    BOS @ MIL  L  62  17  0.785  240   
4   1610612738  0022301134  APR 07, 2024  BOS vs. POR  W  62  16  0.795  240   
..         ...         ...           ...          ... ..  ..  ..    ...  ...   
77  1610612738  0022300136  NOV 04, 2023    BOS @ BKN  W   5   0  1.000  240   
78  1610612738  0022300118  NOV 01, 2023  BOS vs. IND  W   4   0  1.000  240   
79  1610612738  0022300103  OCT 30, 2023    BOS @ WAS  W   3   0  1.000  240   
80  1610612738  0022300080  OCT 27, 2023  BOS vs. MIA  W   2   0  1.000  240   
81  1610612738  0022300065  OCT 25, 2023    BOS @ NYK  W   1   0  1.000  240   

    FGM  ...  FT_PCT  OREB  DREB  REB  

In [21]:
games = scoreboard.ScoreBoard()
print(games.get_dict())


{'meta': {'version': 1, 'request': 'https://nba-prod-us-east-1-mediaops-stats.s3.amazonaws.com/NBA/liveData/scoreboard/todaysScoreboard_00.json', 'time': '2024-10-13 06:37:48.3748', 'code': 200}, 'scoreboard': {'gameDate': '2024-10-13', 'leagueId': '00', 'leagueName': 'National Basketball Association', 'games': [{'gameId': '0012400041', 'gameCode': '20241013/PORSAC', 'gameStatus': 2, 'gameStatusText': 'Q1 :18.4', 'period': 1, 'gameClock': 'PT00M18.40S', 'gameTimeUTC': '2024-10-13T22:00:00Z', 'gameEt': '2024-10-13T18:00:00-04:00', 'regulationPeriods': 4, 'ifNecessary': False, 'seriesGameNumber': '', 'gameLabel': None, 'gameSubLabel': None, 'seriesText': '', 'seriesConference': '', 'poRoundDesc': '', 'gameSubtype': '', 'homeTeam': {'teamId': 1610612758, 'teamName': 'Kings', 'teamCity': 'Sacramento', 'teamTricode': 'SAC', 'wins': 0, 'losses': 0, 'score': 27, 'seed': None, 'inBonus': '0', 'timeoutsRemaining': 5, 'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 27}, {'period': 2,